In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/hackathonw4/Train_Data.csv
/kaggle/input/hackathonw4/Test_Data.csv


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score
from lightgbm import LGBMClassifier

train_df = pd.read_csv("/kaggle/input/hackathonw4/Train_Data.csv")
test_df = pd.read_csv("/kaggle/input/hackathonw4/Test_Data.csv")

train_df = train_df.dropna(subset=['age_group'])
train_df['age_group'] = train_df['age_group'].map({'Adult': 0, 'Senior': 1})

for df in [train_df, test_df]:
    df['glucose_to_insulin_ratio'] = df['LBXGLU'] / (df['LBXIN'] + 1)
    df['glucose_load_ratio'] = df['LBXGLT'] / (df['LBXGLU'] + 1)
    df['insulin_resistant'] = ((df['LBXIN'] > 150) & (df['LBXGLU'] > 120)).astype(int)

selected_features = [
    'BMXBMI',
    'LBXGLU',
    'LBXIN',
    'LBXGLT',
    'glucose_load_ratio',
    'glucose_to_insulin_ratio',
    'insulin_resistant'
]

X = train_df[selected_features]
y = train_df["age_group"]
X_test = test_df[selected_features]

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

num_features = X.select_dtypes(include=['int64', 'float64']).columns

preprocessor = ColumnTransformer(transformers=[
    ("num", Pipeline([
        ('imputer', SimpleImputer(strategy="median")),
        ('scaler', StandardScaler())
    ]), num_features)
])

model = Pipeline(steps=[
    ("preprocessor", preprocessor),
    ("classifier", LGBMClassifier(
        random_state=42,
        class_weight='balanced',
        n_estimators=300,
        learning_rate=0.05,
        max_depth=4,
        num_leaves=15
    ))
])

model.fit(X_train, y_train)

y_pred = model.predict(X_val)
score = f1_score(y_val, y_pred, average='binary')
print("Validation F1 Score:", round(score, 4))

final_predictions = model.predict(X_test)
submission = pd.DataFrame({"age_group": final_predictions})
submission.to_csv("/kaggle/working/submission.csv", index=False)

/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)
/usr/local/lib/python3.11/dist-packages/pandas/core/computation/expressions.py:73: RuntimeWarning: invalid value encountered in greater
  return op(a, b)


[LightGBM] [Info] Number of positive: 251, number of negative: 1310
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003689 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1244
[LightGBM] [Info] Number of data points in the train set: 1561, number of used features: 6
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.500000 -> initscore=-0.000000
[LightGBM] [Info] Start training from score -0.000000
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, be